# 爬取 google map 評論
- [step 1: 載入套件 & 取得 html](#step-1:-載入套件-&-取得-html)
- [step 2: 取得 map url](#step-2:-取得-map-url)
- [step 3: 取得所有 button url (主要有行車路線和官網)](#step-3:-取得所有-button-url-(主要有行車路線和官網))
- [step 4: 其他資訊](#step-4:-其他資訊)
- [step 5: 處理其他人也搜尋](#step-5:-處理其他人也搜尋)
- [step 6: 使用 api 處理動態增加的評論](#step-6:-使用-api-處理動態增加的評論)

# 載入套件

In [1]:
#讀取網頁原始碼
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
import urllib

# step 1: 將 key word 轉換成搜尋 url & 確認該 url 頁面狀態

In [2]:
def get_search_url(target):
    # string to url encoding
    tar_urlencode = urllib.parse.quote(target)

    return 'https://www.google.com/search?q={}'.format(tar_urlencode)

In [3]:
def get_header():
    return {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'}

In [4]:
target = '陽明山'
# target = "冷水坑"
# target = '八仙洞'
url = get_search_url(target)
print(url)

https://www.google.com/search?q=%E9%99%BD%E6%98%8E%E5%B1%B1


## 確認當下取得的取得的網頁狀態

In [5]:
html = requests.get(url, headers=get_header())
if html.status_code == 200:
    print('success!')

success!


In [6]:
# html 解析器官方建議使用 lxml ，會比預設的 html.parser 快。
soup = BeautifulSoup(html.text, 'lxml') # soup = BeautifulSoup(html.text, 'html.parser')

In [7]:
# 是否包含多個搜尋結果(通常景點範圍較大)
soup.find("div", class_="xERobd")

In [8]:
# 蒐集本次爬從所有資訊
info_dt = {}

# step 2: 取得 map url
---
![](https://i.imgur.com/EmfoIgD.png)

In [9]:
map_div_tag = soup.find('img',class_="lu-fs")

In [10]:
map_div_tag

In [11]:
# 這代表 <img> 這區塊 有26個比它大的層包住它
len(list(map_div_tag.parents))

24

In [12]:
outer_level = list(map_div_tag.parents)[0]

In [13]:
locat_map_url = 'https://www.google.com{}'.format(outer_level['data-url'])
info_dt['google_map_url'] = locat_map_url
print(locat_map_url)

https://www.google.com/maps/place/%E5%85%AB%E4%BB%99%E6%B4%9E/@23.3971669,121.476676,15z/data=!4m2!3m1!1s0x0:0x2054d2f4f6524d10?sa=X




# step 3: 取得所有 button url (主要有行車路線和官網)

---

![](https://i.imgur.com/7FzPpt5.png)

In [14]:
# 所有 button 的文字
button_tag_ls = soup.find_all('a',class_="ab_button")

In [15]:
button_tag_dt = {i.text:i for i in button_tag_ls}

In [16]:
if '網站'in button_tag_dt:
    info_dt['official_web'] = button_tag_dt['網站']['href']
    print(button_tag_dt['網站']['href'])

http://www.eastcoast-nsa.gov.tw/zh-tw/Attractions/Detail/37


In [17]:
if '行車路線'in button_tag_dt:
    info_dt['google_map_route'] = button_tag_dt['行車路線']['data-url']
    print('https://www.google.com{}'.format(button_tag_dt['行車路線']['data-url']))

https://www.google.com/maps/dir//%E5%85%AB%E4%BB%99%E6%B4%9E/data=!4m6!4m5!1m1!4e2!1m2!1m1!1s0x346f5b65ac0a14ff:0x2054d2f4f6524d10?sa=X


# step 4: 其他資訊
---
![](https://i.imgur.com/9liWqVC.png)

In [18]:
# 評分
star_tag = soup.find(class_="Aq14fc")
info_dt['星星數'] = star_tag.text
print(star_tag.text)

3.7


In [19]:
# 共幾則評論，可供後續爬取權評論用
comment_count = soup.find(class_="hqzQac")
print(comment_count.text)
info_dt['總評論筆數'] = comment_count.text
info_dt['總評論筆數'] = re.match('\d\W?\d+',info_dt['總評論筆數']).group(0).replace(',','')

2,168 則 Google 評論


In [20]:
# 主要資訊
main_info = soup.find("h2",class_="bNg8Rb",text="說明")
info_dt['景點簡介'] = main_info.next_sibling.text
print(main_info.next_sibling.text)

八仙洞遺址，俗稱八仙洞，台灣旅遊景點、考古遺址，位於台東縣長濱鄉省道台11線公路旁，為舊石器時代之長濱文化的代表。在2006年，被中華民國政府指定為國定遺址。


In [21]:
info_ls = soup.find_all(class_="mod")

In [22]:
target_title = ['地址','開放時間','登錄名稱','位置','電話','人們在此停留的時間']
for i in info_ls:
    t = i.text
    for tar in target_title:
        if t.find(tar) != -1:
            info_dt.update({tar:i.text.split('：')[-1].strip()})

In [23]:
info_dt

{'google_map_url': 'https://www.google.com/maps/place/%E5%85%AB%E4%BB%99%E6%B4%9E/@23.3971669,121.476676,15z/data=!4m2!3m1!1s0x0:0x2054d2f4f6524d10?sa=X',
 'official_web': 'http://www.eastcoast-nsa.gov.tw/zh-tw/Attractions/Detail/37',
 'google_map_route': '/maps/dir//%E5%85%AB%E4%BB%99%E6%B4%9E/data=!4m6!4m5!1m1!4e2!1m2!1m1!1s0x346f5b65ac0a14ff:0x2054d2f4f6524d10?sa=X',
 '星星數': '3.7',
 '總評論筆數': '2168',
 '景點簡介': '八仙洞遺址，俗稱八仙洞，台灣旅遊景點、考古遺址，位於台東縣長濱鄉省道台11線公路旁，為舊石器時代之長濱文化的代表。在2006年，被中華民國政府指定為國定遺址。',
 '地址': '962台東縣長濱鄉三間村水母丁1-4號',
 '開放時間': '週三08:30星期二08:30–17:00星期三08:30–17:00星期四08:30–17:00星期五08:30–17:00星期六08:30–17:00星期日08:30–17:00星期一08:30–17:00提出修改建議無法新增這個檔案，請確認這張相片是有效的檔案。',
 '登錄名稱': '八仙洞遺址',
 '位置': '中華民國臺東縣長濱鄉',
 '電話': '08 988 1418',
 '人們在此停留的時間': '人們在此停留的時間通常不會超過 45 分鐘'}

# step 5: 處理其他人也搜尋
---

這邊需要先取得`查看更多項目` 的 url ，才能轉去有完整景點的網站
![](https://i.imgur.com/6xNrXdl.png)

In [24]:
other_search = soup.find(class_="VLkRKc")

In [25]:
other_search_url = 'https://www.google.com{}'.format(other_search.a['href'])
info_dt['other_search_url'] = other_search_url
print(other_search_url)

https://www.google.com/search?q=%E5%85%AB%E4%BB%99%E6%B4%9E%E9%81%BA%E5%9D%80&stick=H4sIAAAAAAAAAONgFuLWT9c3NDUtMTBLS1FC5mipZSdb6ZcUJZal5uiX5JcWZRaXxCeWAAWSSzLz86yKM1NSyxMrixex8j9tXf1k98xnW-a9bNz1dG7DDlZGAFY_luBZAAAA&sa=X&ved=2ahUKEwiymtG-zfnoAhUmw4sBHQxtBKwQMSgAMBx6BAgPEAE


### 上面網址可以來到下圖
因為它的資料都是預載入，所以可以直接從 html 解析得到，不需要動態或是模仿api

![](https://i.imgur.com/5DC5tRI.png)

In [26]:
html_other = requests.get(other_search_url, headers = get_header())

if html_other.status_code == 200:
    print('success!')
    
soup_other = BeautifulSoup(html_other.text, 'lxml')

other_search_ls = soup_other.find_all("a",class_="llgymd")

success!


In [27]:
if len(other_search_ls) ==0:
    print('error!')
else:
    print('success!')

success!


In [28]:
other_search_dt = {spot['aria-label']:'https://www.google.com{}'.format(spot['href']) for spot in other_search_ls}

In [29]:
info_dt['other_search_result'] = other_search_dt

In [30]:
info_dt

{'google_map_url': 'https://www.google.com/maps/place/%E5%85%AB%E4%BB%99%E6%B4%9E/@23.3971669,121.476676,15z/data=!4m2!3m1!1s0x0:0x2054d2f4f6524d10?sa=X',
 'official_web': 'http://www.eastcoast-nsa.gov.tw/zh-tw/Attractions/Detail/37',
 'google_map_route': '/maps/dir//%E5%85%AB%E4%BB%99%E6%B4%9E/data=!4m6!4m5!1m1!4e2!1m2!1m1!1s0x346f5b65ac0a14ff:0x2054d2f4f6524d10?sa=X',
 '星星數': '3.7',
 '總評論筆數': '2168',
 '景點簡介': '八仙洞遺址，俗稱八仙洞，台灣旅遊景點、考古遺址，位於台東縣長濱鄉省道台11線公路旁，為舊石器時代之長濱文化的代表。在2006年，被中華民國政府指定為國定遺址。',
 '地址': '962台東縣長濱鄉三間村水母丁1-4號',
 '開放時間': '週三08:30星期二08:30–17:00星期三08:30–17:00星期四08:30–17:00星期五08:30–17:00星期六08:30–17:00星期日08:30–17:00星期一08:30–17:00提出修改建議無法新增這個檔案，請確認這張相片是有效的檔案。',
 '登錄名稱': '八仙洞遺址',
 '位置': '中華民國臺東縣長濱鄉',
 '電話': '08 988 1418',
 '人們在此停留的時間': '人們在此停留的時間通常不會超過 45 分鐘',
 'other_search_url': 'https://www.google.com/search?q=%E5%85%AB%E4%BB%99%E6%B4%9E%E9%81%BA%E5%9D%80&stick=H4sIAAAAAAAAAONgFuLWT9c3NDUtMTBLS1FC5mipZSdb6ZcUJZal5uiX5JcWZRaXxCeWAAWSSzLz86yKM1NSyxMrixex8j9tXf1k98xnW-a9bNz1dG7DDlZ

## step 6: 使用 api 處理動態增加的評論
---
![](https://i.imgur.com/NmXFTOd.png)

In [9]:
# 取得 feature id
buttom_ls = soup.find_all(role="button")
comment_buttom = [i for i in buttom_ls if i.text=='查看所有 Google 評論'][0]
feature_id = urllib.parse.quote(comment_buttom['data-fid'])
print(feature_id)

0x3442adf824cfc74b%3A0x5c0be3a34fc3b079


In [10]:
# 取得 ei
ei_ls = soup.find_all("input", type="hidden")

def find_ei(ei_ls:list):
    for i in ei_ls:
        try:
            if i['name'] == "ei": return i['value']
        except:
            continue
            
ei = find_ei(ei_ls)
print(ei)

YZFDX4GSOdHomAXz6JXoAw


In [11]:
# 生成 api 網址
def get_api_url(idx:int, fid:str, ei:str):
    api_url = "https://www.google.com/async/reviewDialog?ei={}&yv=3&async=feature_id:{},review_source:All%20reviews,sort_by:qualityScore,start_index:{},is_owner:false,filter_text:,associated_topic:,next_page_token:,async_id_prefix:,_pms:s,_fmt:pc"\
    .format(ei, fid, idx)
    
    return api_url

In [12]:
def get_comment(url:str, debug=True):
    html_coment = requests.get(url, headers = get_header(), stream=True)

    if (html_coment.status_code == 200)&(debug):
        print('success!')

    soup_coment = BeautifulSoup(html_coment.text, 'lxml')
    tmp = soup_coment.find_all('span',jscontroller="i6OOIc")
    comment_ls = []
    for i in tmp:
        comment_ls.append(i.text)
    
    return comment_ls

In [13]:
api_url = get_api_url(0, fid=feature_id, ei=ei)
html_coment = requests.get(api_url, headers = get_header(), stream=True)
soup_coment = BeautifulSoup(html_coment.text, 'lxml')
tmp = soup_coment.find_all('div', jscontroller="e6Mltc") # 整筆評論主要物件

In [15]:
tmp[0]

<div class="WMbnJf gws-localreviews__google-review" jsaction="rcuQ6b:npT2md" jscontroller="e6Mltc" jsdata="iVOPBe;;AWDNlc"><a class="fl" href="https://www.google.com/maps/contrib/102532542882564690524?hl=zh-Hant-TW&amp;sa=X&amp;ved=2ahUKEwia_d-OzLPrAhXlKqYKHaeHCdQQvfQBegQIARAU" style="cursor:pointer" tabindex="-1"><img alt="Yitine Hange" class="lDY1rd" src="https://lh4.googleusercontent.com/-ZTagYVs27v4/AAAAAAAAAAI/AAAAAAAAAAA/AMZuucmWzerYUpm6uVNXVXHDBLVHQowBXA/s40-c0x00000000-cc-rp/photo.jpg"/></a><div class="jxjCjc"><div class="TSUbDb" style="display:block"><a href="https://www.google.com/maps/contrib/102532542882564690524?hl=zh-Hant-TW&amp;sa=X&amp;ved=2ahUKEwia_d-OzLPrAhXlKqYKHaeHCdQQvvQBegQIARAV">Yitine Hange</a></div><div class="FGlxyd" style="display:block"><a class="Msppse" href="https://www.google.com/maps/contrib/102532542882564690524?hl=zh-Hant-TW&amp;sa=X&amp;ved=2ahUKEwia_d-OzLPrAhXlKqYKHaeHCdQQvvQBegQIARAW"><span class="A503be">4 則評論</span></a></div><div style="vertical-a

In [16]:
[i.text for i in soup_coment.find_all('div', class_="TSUbDb")]

['Yitine Hange',
 'wallace yang',
 '張敏瓊',
 '何良容',
 '紙上談餅',
 '北有涼城',
 '夜泊秦淮',
 '臥龍坡',
 '草莓味萌軟妹子耶',
 '林月秀']

In [17]:
[i.text for i in soup_coment.find_all('span', jsname="CMh1ye")] # 評論案讚數

['0', '0', '0', '0', '1', '0', '0', '0', '0', '0']

In [18]:
[i.text for i in soup_coment.find_all('span', jscontroller="P7L8k")] # 評論案讚數

['海芋季非常推薦一定要來！！！（不建議假日 塞爆）空氣清新  風景宜人 ，超適合帶二寶來放電價格也不貴',
 '大屯山階梯很長，要走上去有累。但無敵美景會讓你忘記了辛苦。值了...',
 '雙北近郊的國家公園，有超多步道可以走，且規劃良好，停車方便，風景美不勝收，只是假日人潮太多，要注意交通管制，可以多利用大眾運輸工具',
 '這是一個遠足的好地方。風景令人嘆為觀止。如果你不喜歡人群，就不要在周末和國慶假期來。如果下雨，就不要去：你會渾身濕透的。因為風很大，傘撐不了多久！享受！',
 '做為台北市的國家公園當之無愧，有著許多綠意盎然的步道，空氣品質好，蟲鳴鳥叫，遠離塵囂療癒身心，真的是台北的寶山',
 '我們租了一輛車去那裡，並且很幸運地可以在漫長的假期開始前進行参觀。我們的司機告訴我們，節假日期間會有交通管制，所以我們可能不能進入。由於已經是櫻花盛開的季節，這裏擠滿了遊客，但我們仍然很喜歡這個地方，很享受這裏的好天氣。',
 '幸好台灣的朋友帶我們去陽明山，碰巧遇上茶花展，各式各樣的花，美得令眼睛發亮，頭一次看到這麼多又美的茶花，還有鬱金香，風信子⋯',
 '陽明山位於台北，山上有唔少靚景，花季時更會百花齊開。因為火山原因，依度亦形成地熱湖同硫酸谷依兩個特別景點。',
 '好逛，當地的花季不能不去看看，一年四季有著不同的風格，整座陽明山值得遊客花著一個星期來做旅遊規劃，勤天崗、竹子湖、文化大學、特色咖啡廳...等，不要再跟團請自己來遊玩，租個機車汽車慢慢欣賞整條路的風光',
 '油綠綠的大草原，看的到風在跑，風吹雲動，地上的光影也跟著移動，像調皮光腳丫的孩子滿山跑，風拂過綠草，拂過樹梢，心兒也跟著輕飄飄，樂逍遙🤭']

In [19]:
[i.text for i in soup_coment.find_all('span', class_="dehysf")]

['3 天前',
 '1 週前',
 '1 週前',
 '1 週前',
 '3 週前',
 '1 個月前',
 '4 週前',
 '3 週前',
 '3 週前',
 '3 週前']

In [20]:
[i['aria-label'][3:6] for i in soup_coment.find_all('span', class_="Fam1ne EBe2gf")]

['5.0', '4.0', '5.0', '4.0', '5.0', '4.0', '5.0', '4.0', '5.0', '4.0']

In [74]:
[i.a['href'] for i in soup_coment.find_all('div', class_="TSUbDb")] # 評論者頁面

['https://www.google.com/maps/contrib/114962840696865089902?hl=zh-Hant-TW&sa=X&ved=2ahUKEwiz4rDV4qbrAhXXyYsBHfORBr8QvvQBegQIARAV',
 'https://www.google.com/maps/contrib/105068324802608402315?hl=zh-Hant-TW&sa=X&ved=2ahUKEwiz4rDV4qbrAhXXyYsBHfORBr8QvvQBegQIARAe',
 'https://www.google.com/maps/contrib/114563997494302504268?hl=zh-Hant-TW&sa=X&ved=2ahUKEwiz4rDV4qbrAhXXyYsBHfORBr8QvvQBegQIARAn',
 'https://www.google.com/maps/contrib/115192736069800710885?hl=zh-Hant-TW&sa=X&ved=2ahUKEwiz4rDV4qbrAhXXyYsBHfORBr8QvvQBegQIARAw',
 'https://www.google.com/maps/contrib/115868470606340691124?hl=zh-Hant-TW&sa=X&ved=2ahUKEwiz4rDV4qbrAhXXyYsBHfORBr8QvvQBegQIARA4',
 'https://www.google.com/maps/contrib/101567502028557646758?hl=zh-Hant-TW&sa=X&ved=2ahUKEwiz4rDV4qbrAhXXyYsBHfORBr8QvvQBegQIARBB',
 'https://www.google.com/maps/contrib/116055145247201351700?hl=zh-Hant-TW&sa=X&ved=2ahUKEwiz4rDV4qbrAhXXyYsBHfORBr8QvvQBegQIARBJ',
 'https://www.google.com/maps/contrib/105672346162619420300?hl=zh-Hant-TW&sa=X&ved=

In [76]:
[(i.text.find('在地嚮導')!=-1) for i in soup_coment.find_all('div', jscontroller="e6Mltc")]

[True, True, False, False, False, True, False, False, False, False]

In [21]:
a = [i.text for i in soup_coment.find_all('div', class_="TSUbDb")]
b = [(i.text.find('在地嚮導')!=-1) for i in soup_coment.find_all('div', jscontroller="e6Mltc")]
c = [i.text for i in soup_coment.find_all('span', jscontroller="P7L8k")] # 評論案讚數

In [22]:
list(zip(a,b,c))

[('Yitine Hange', False, '海芋季非常推薦一定要來！！！（不建議假日 塞爆）空氣清新  風景宜人 ，超適合帶二寶來放電價格也不貴'),
 ('wallace yang', True, '大屯山階梯很長，要走上去有累。但無敵美景會讓你忘記了辛苦。值了...'),
 ('張敏瓊',
  True,
  '雙北近郊的國家公園，有超多步道可以走，且規劃良好，停車方便，風景美不勝收，只是假日人潮太多，要注意交通管制，可以多利用大眾運輸工具'),
 ('何良容',
  False,
  '這是一個遠足的好地方。風景令人嘆為觀止。如果你不喜歡人群，就不要在周末和國慶假期來。如果下雨，就不要去：你會渾身濕透的。因為風很大，傘撐不了多久！享受！'),
 ('紙上談餅', True, '做為台北市的國家公園當之無愧，有著許多綠意盎然的步道，空氣品質好，蟲鳴鳥叫，遠離塵囂療癒身心，真的是台北的寶山'),
 ('北有涼城',
  False,
  '我們租了一輛車去那裡，並且很幸運地可以在漫長的假期開始前進行参觀。我們的司機告訴我們，節假日期間會有交通管制，所以我們可能不能進入。由於已經是櫻花盛開的季節，這裏擠滿了遊客，但我們仍然很喜歡這個地方，很享受這裏的好天氣。'),
 ('夜泊秦淮',
  False,
  '幸好台灣的朋友帶我們去陽明山，碰巧遇上茶花展，各式各樣的花，美得令眼睛發亮，頭一次看到這麼多又美的茶花，還有鬱金香，風信子⋯'),
 ('臥龍坡', False, '陽明山位於台北，山上有唔少靚景，花季時更會百花齊開。因為火山原因，依度亦形成地熱湖同硫酸谷依兩個特別景點。'),
 ('草莓味萌軟妹子耶',
  False,
  '好逛，當地的花季不能不去看看，一年四季有著不同的風格，整座陽明山值得遊客花著一個星期來做旅遊規劃，勤天崗、竹子湖、文化大學、特色咖啡廳...等，不要再跟團請自己來遊玩，租個機車汽車慢慢欣賞整條路的風光'),
 ('林月秀',
  True,
  '油綠綠的大草原，看的到風在跑，風吹雲動，地上的光影也跟著移動，像調皮光腳丫的孩子滿山跑，風拂過綠草，拂過樹梢，心兒也跟著輕飄飄，樂逍遙🤭')]

### 測試 start index 為 0

In [48]:
api_url = get_api_url(0, fid=feature_id, ei=ei)

In [51]:
get_comment(api_url)

success!


[]

### 測試 start index 為 9

可以看到首個(index=9)，為上面的最後一個，可以知道在 url 此 api 可以由 start_index 決定回傳的首個評論。

In [37]:
api_url = get_api_url(9, fid=feature_id, ei=ei)

In [38]:
get_comment(api_url)

success!


['2019/12/08再度走訪台東濱海的八仙洞，才知道八仙洞原來是舊石器時代遺址，印象還停留在二十年前恐怖的神壇洞⋯⋯原來台東縣政府花了很多年的時間請走佔據八仙洞的神祇，逐步收回考古遺跡⋯目前還在維修中！待重新開放時，定要再訪潮音洞！細細聆聽八仙洞澎湃的自然音～',
 '路過值得下車走一趟，有小小風景及可聽到浪濤的回音，很特別',
 '目前只有靈巖洞開放，上面的海蝕洞都應步道整修不開放，十分可惜，只能來這聽海哭的聲音了',
 '自然風景及歷史古蹟的景點，適合小朋友及年長者踏青旅遊的地方。',
 '有些地方因為有安全疑慮封起來 但這是很有歷史意義的遺址 推薦大家來這裡走走',
 '2019.10.12目前步道封閉了，海蝕洞也因落石而封閉入口，不知何時會修好。全台灣最大、最壯闊的海蝕洞。 ⋯⋯更多2019.10.12目前步道封閉了，海蝕洞也因落石而封閉入口，不知何時會修好。全台灣最大、最壯闊的海蝕洞。「小帖步」1. 有停車場，要付費，但不貴，小客車30元，大客車100元。停車場有上下兩座，我通常都停下面、商店街前面那座，但位置不多，大概總共二十個車位、兩排而已，且是死巷，動線不好通行，所以人多時可能只能停上面那座。2. 八仙洞應該有八個，但登山步道最近因落石而在整修，入口被封住了，不過最大的那個靈石洞還可以接近、參觀，只是入口處也因落石而被圍起來，以前是可以直接到最裡面的。3. 靈石洞又稱潮音洞，在裡面可以聽到轟隆隆的浪潮回音，建議可以坐下來，感受那視覺與聽覺得以壯麗感。',
 '登山步道整修，只有靈岩洞可看，其他七仙洞無法看到，可惜。',
 '2/13去,步道整修只能看到一個洞',
 '目前景點都在修繕中，但仍收取停車費卻未告知...',
 '終於把違章雜亂拆光了，值得一遊的天然海蝕洞。']

### 獲取全部評論
因為 google 會移除不當內容，所以得到的評論筆數會與總比數有落差
![](https://i.imgur.com/wk00FDm.png)

In [39]:
comment_count = int(info_dt['總評論筆數'])
crawler_time = int(comment_count/10)+1
print(comment_count)

2168


In [40]:
comment_result_ls = []
for i in range(crawler_time):
    idx = i*10
    api_url = get_api_url(idx, fid=feature_id, ei=ei)
    comment_respond = get_comment(api_url, debug=False)
    if comment_respond == ['' for i in range(10)]:
        break
    comment_result_ls.extend(comment_respond)

In [41]:
comment_result_ls

['洞穴收回公有，乾淨多了，但8洞只有1洞可參觀（但不得入內） 其餘因步道修整不得參觀，可惜！',
 '第二次停留此地,小車停車費25元相對其他景點較為便宜,走上去看海十分愜意,廁所打掃十分清潔 加分.',
 '跟小時候的八仙洞不一樣把一些神請走，現在去看格外乾淨東西少了許多，個人對這類事情沒意見但因為長年吹海風蠻容易有落石，希望可以好好的整修一下。但遊客好像跟以前比起來少上許多!!!',
 '所有棧道、洞穴全部封閉，停車場還是照收費60元進去，不知什麼意思，等維修好再開放吧',
 '海潮洞傳出海浪聲的回音，很適合停車下來子走走了解台灣豐富的史前文明與聽海潮聲放鬆心靈😌',
 '這是舊石器時代長濱文化的發現地，也是臺灣目前最早的史前文化遺址。唯恐落石傷人，所以在洞外觀看即可，切勿深入。',
 '因為年久風化，不時會有落石，現已把佛像遷往他處了，不過還是可以欣賞自然景觀',
 '騎著車，暢遊在台11線，右邊即是蔚藍的太平洋，走在山壁旁，感受著大自然的鬼斧神工，非常舒服，可惜當地政府沒有好好規劃導覽或是舉辦活動促進觀光。',
 '台灣最早的人類遺址 三萬年前就有人居住 可惜很多區域因為安全考量沒有開放',
 '2019/12/08再度走訪台東濱海的八仙洞，才知道八仙洞原來是舊石器時代遺址，印象還停留在二十年前恐怖的神壇洞⋯⋯原來台東縣政府花了很多年的時間請走佔據八仙洞的神祇，逐步收回考古遺跡⋯目前還在維修中！待重新開放時，定要再訪潮音洞！細細聆聽八仙洞澎湃的自然音～',
 '路過值得下車走一趟，有小小風景及可聽到浪濤的回音，很特別',
 '目前只有靈巖洞開放，上面的海蝕洞都應步道整修不開放，十分可惜，只能來這聽海哭的聲音了',
 '自然風景及歷史古蹟的景點，適合小朋友及年長者踏青旅遊的地方。',
 '有些地方因為有安全疑慮封起來 但這是很有歷史意義的遺址 推薦大家來這裡走走',
 '2019.10.12目前步道封閉了，海蝕洞也因落石而封閉入口，不知何時會修好。全台灣最大、最壯闊的海蝕洞。 ⋯⋯更多2019.10.12目前步道封閉了，海蝕洞也因落石而封閉入口，不知何時會修好。全台灣最大、最壯闊的海蝕洞。「小帖步」1. 有停車場，要付費，但不貴，小客車30元，大客車100元。停車場有上下兩座，我通常都停下面、商店街前面那座，但位置不多，大概總共二十個車位、兩排而已，且是死巷

In [43]:
info_dt['comment'] = comment_result_ls

## step 7: 爬取時段

In [44]:
open_day = soup.find_all('td', class_="SKNSIb")

In [46]:
open_time = {i.text:i.next_sibling.text for i in open_day}

In [47]:
info_dt['營業時間'] = open_time
print(open_time)

{'星期二': '08:30–17:00', '星期三': '08:30–17:00', '星期四': '08:30–17:00', '星期五': '08:30–17:00', '星期六': '08:30–17:00', '星期日': '08:30–17:00', '星期一': '08:30–17:00'}


In [48]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [49]:
browser = webdriver.Chrome('../chromedriver.exe')
browser.get(info_dt['google_map_url'])

In [56]:
htmlstring = browser.page_source
browser.close()

In [57]:
soup_map = BeautifulSoup(htmlstring, 'lxml')

In [58]:
soup_map_clear = soup_map.find(class_="section-popular-times-container")

In [59]:
soup_map_d_ls = [i for i in list(soup_map_clear.children) if i !=' ']

In [92]:
result_dt = {}
for day, item in enumerate(soup_map_d_ls):
    soup_map_t_ls = [i['aria-label'] for i in list(item.descendants) if (str(i).find('繁忙程度')!=-1) & (str(i).find('section-popular-times-graph')==-1)]
    result_dt[day] = {i.split(' ')[0]:i.split(' ')[2].replace('。','') for i in soup_map_t_ls}

In [93]:
result_dt

{0: {'6時': '0%',
  '7時': '0%',
  '8時': '1%',
  '9時': '6%',
  '10時': '26%',
  '11時': '70%',
  '12時': '100%',
  '13時': '90%',
  '14時': '70%',
  '15時': '52%',
  '16時': '30%',
  '17時': '0%',
  '18時': '0%',
  '19時': '0%',
  '20時': '0%',
  '21時': '0%',
  '22時': '0%',
  '23時': '0%'},
 1: {'6時': '0%',
  '7時': '0%',
  '8時': '3%',
  '9時': '10%',
  '10時': '23%',
  '11時': '40%',
  '12時': '54%',
  '13時': '58%',
  '14時': '49%',
  '15時': '33%',
  '16時': '19%',
  '17時': '0%',
  '18時': '0%',
  '19時': '0%',
  '20時': '0%',
  '21時': '0%',
  '22時': '0%',
  '23時': '0%'},
 2: {'6時': '0%',
  '7時': '0%',
  '8時': '3%',
  '9時': '9%',
  '10時': '21%',
  '11時': '38%',
  '12時': '54%',
  '13時': '62%',
  '14時': '56%',
  '15時': '40%',
  '16時': '23%',
  '17時': '0%',
  '18時': '0%',
  '19時': '0%',
  '20時': '0%',
  '21時': '0%',
  '22時': '0%',
  '23時': '0%'},
 3: {'6時': '0%',
  '7時': '0%',
  '8時': '3%',
  '9時': '9%',
  '10時': '20%',
  '11時': '37%',
  '12時': '52%',
  '13時': '58%',
  '14時': '48%',
  '15時': '32%',
  '16時': '16

In [100]:
pd.DataFrame(info_dt['時間人潮'])

,0,1,2,3,4,5,6
6時,0%,0%,0%,0%,0%,0%,0%
7時,0%,0%,0%,0%,0%,0%,0%
8時,1%,3%,3%,3%,2%,4%,2%
9時,6%,10%,9%,9%,7%,12%,9%
10時,26%,23%,21%,20%,19%,25%,30%
11時,70%,40%,38%,37%,33%,34%,55%
12時,100%,54%,54%,52%,45%,35%,63%
13時,90%,58%,62%,58%,51%,41%,58%
14時,70%,49%,56%,48%,54%,56%,67%
15時,52%,33%,40%,32%,48%,55%,77%


In [94]:
info_dt['時間人潮'] = result_dt

In [95]:
df = pd.DataFrame.from_dict(info_dt, orient="index")

In [96]:
df.T.to_json('{}_result.json'.format(target))

In [97]:
pd.DataFrame(info_dt['comment'], columns=['評論']).to_excel('{}_comment_result.xlsx'.format(target))

In [98]:
df = pd.DataFrame.from_dict(info_dt['other_search_result'], orient="index").reset_index()

In [99]:
# 解決 excel 不能存長網址，https://stackoverflow.com/questions/35440528/how-to-save-in-xlsx-long-url-in-cell-using-pandas
writer = pd.ExcelWriter('{}_relate_search_result.xlsx'.format(target), engine='xlsxwriter',options={'strings_to_urls': False})
df.to_excel(writer)
writer.close()